Taking the top 500 albums of all time as ranked by Rolling Stone and finding the mean value after performing sentiment analysis on their lyrics

In [1]:
!pip install lyricsgenius
!pip install vaderSentiment
!pip install nltk
import nltk
import json
import requests
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from lyricsgenius import Genius
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import numpy as np 
import requests
from io import BytesIO 
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os
from dotenv import load_dotenv